In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [0]:
rec_df = pd.read_csv("compass/propublica_data_for_fairml.csv")

In [3]:
rec_df.columns

Index(['Two_yr_Recidivism', 'Number_of_Priors', 'score_factor',
       'Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'African_American',
       'Asian', 'Hispanic', 'Native_American', 'Other', 'Female',
       'Misdemeanor'],
      dtype='object')

In [0]:
X = rec_df[['Number_of_Priors', 'score_factor', 'Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'African_American', 'Asian', 'Hispanic', 'Native_American', 'Other', 'Female', 'Misdemeanor']].values
y = rec_df['Two_yr_Recidivism'].values

In [5]:
y.shape

(6172,)

In [6]:
X.shape

(6172, 11)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline



In [0]:
#from_numpy takes a numpy element and returns torch.tensor
X = torch.from_numpy(X_train).type(torch.FloatTensor)
y = torch.from_numpy(y_train).type(torch.LongTensor)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
#our class must extend nn.Module
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier,self).__init__()
        #Our network consists of 3 layers. 1 input, 1 hidden and 1 output layer
        #This applies Linear transformation to input data. 
        self.fc1 = nn.Linear(11,5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)

            
        
        #This applies linear transformation to produce output data
        self.fc4 = nn.Linear(3,2)
        
    #This must be implemented
    def forward(self,x):
        #Output of the first layer
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.fc3(x)
        x = F.tanh(x)
        x = self.fc4(x)
        #This produces output
        return x
        
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        pred = F.softmax(self.forward(x))
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [0]:
#Initialize the model        
model = MyClassifier()
#Define loss criterion
criterion = nn.CrossEntropyLoss()
#Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [12]:
#Number of epochs
epochs = 10000
#List to store losses
losses = []
for i in range(epochs):
    #Precit the output for Given input
    y_pred = model.forward(X)
    #Compute Cross entropy loss
    loss = criterion(y_pred,y)
    #Add loss to the list
    losses.append(loss.item())
    print(loss.item())
    #Clear the previous gradients
    optimizer.zero_grad()
    #Compute gradients
    loss.backward()
    #Adjust weights
    optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0.7280611395835876
0.7177808880805969
0.7085793614387512
0.7002817988395691
0.6927876472473145
0.6860620975494385
0.6800511479377747
0.6746208667755127
0.6695986390113831
0.6647933721542358
0.6600261330604553
0.6551423668861389
0.6500698328018188
0.6448407769203186
0.6395996809005737
0.634558916091919
0.6299529075622559
0.6259370446205139
0.6225621700286865
0.6197466254234314
0.6173595190048218
0.61527019739151
0.6134065389633179
0.6117212772369385
0.6102160811424255
0.6088893413543701
0.6077382564544678
0.6067243218421936
0.6057806611061096
0.6048478484153748
0.6038928627967834
0.6029254198074341
0.601997971534729
0.6011555790901184
0.6004577875137329
0.5999197363853455
0.5995423197746277
0.599330484867096
0.5992765426635742
0.5993589162826538
0.599503219127655
0.5995894074440002
0.5995222926139832
0.5993058085441589
0.5989816188812256
0.5985916256904602
0.5981841087341309
0.5977884531021118
0.597447395324707
0.5971856713294983
0.5970216393470764
0.5969364047050476
0.5968962907791138


In [13]:
from sklearn.metrics import accuracy_score
print(model)
print(accuracy_score(model.predict(X),y))

MyClassifier(
  (fc1): Linear(in_features=11, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=2, bias=True)
)
0.703868746202147


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


SAME THING IN KERAS :)

In [14]:
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
input_shape = (11,)

In [0]:
input = Input(shape=input_shape)
x = Dense(64, activation='tanh')(input)
x = Dense(32, activation='tanh')(x)
x = Dense(16, activation='tanh')(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input, x, name='model')

In [0]:
from keras import optimizers
learning_rate = 0.01
model.compile(optimizer=optimizers.Adam(lr=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])

In [57]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 11)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                768       
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 17        
Total params: 3,393
Trainable params: 3,393
Non-trainable params: 0
_________________________________________________________________


In [0]:
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [58]:
model.fit(x=X_train, y=y_train, batch_size=None, epochs=1000, validation_split=0.2)

Train on 3949 samples, validate on 988 samples
Epoch 1/1000
3949/3949 [==============================] - 1s 278us/step - loss: 0.6168 - acc: 0.6716 - val_loss: 0.6230 - val_acc: 0.6711
Epoch 2/1000
3949/3949 [==============================] - 0s 56us/step - loss: 0.6055 - acc: 0.6825 - val_loss: 0.6235 - val_acc: 0.6670
Epoch 3/1000
3949/3949 [==============================] - 0s 54us/step - loss: 0.6038 - acc: 0.6799 - val_loss: 0.6162 - val_acc: 0.6711
Epoch 4/1000
3949/3949 [==============================] - 0s 51us/step - loss: 0.6050 - acc: 0.6873 - val_loss: 0.6230 - val_acc: 0.6609
Epoch 5/1000
3949/3949 [==============================] - 0s 51us/step - loss: 0.6002 - acc: 0.6842 - val_loss: 0.6295 - val_acc: 0.6538
Epoch 6/1000
3949/3949 [==============================] - 0s 53us/step - loss: 0.5977 - acc: 0.6903 - val_loss: 0.6663 - val_acc: 0.6741
Epoch 7/1000
3949/3949 [==============================] - 0s 52us/step - loss: 0.5998 - acc: 0.6855 - val_loss: 0.6415 - val_acc: 

In [59]:
model.evaluate(x=X_test, y=y_test)

1235/1235 [==============================] - 0s 32us/step


[0.6333225513759412, 0.6631578949298936]

In [60]:
model.metrics_names

['loss', 'acc']

In [61]:
X_test.shape

(1235, 11)

In [65]:
result = model.predict(pd.DataFrame(X_test).iloc[[0]].to_numpy())

result[0][0]

0.7956019

In [66]:
y_pred = model.predict(X_test)
type(y_pred)

numpy.ndarray

In [67]:
import collections
collections.Counter(y_test)

Counter({0: 683, 1: 552})

In [77]:
#Change the threshhold here


y_pred=np.array([0 if i[0] < 0.70 else 1 for i in model.predict(X_test)])
collections.Counter(y_pred)

Counter({0: 910, 1: 325})

In [0]:
from sklearn import metrics
matrix = metrics.confusion_matrix(y_test, y_pred)

In [79]:
print(matrix)
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[[578 105]
 [332 220]]
              precision    recall  f1-score   support

           0       0.64      0.85      0.73       683
           1       0.68      0.40      0.50       552

    accuracy                           0.65      1235
   macro avg       0.66      0.62      0.61      1235
weighted avg       0.65      0.65      0.63      1235

Accuracy: 0.6461538461538462
